# Using Jupyter on your cluster

## Setup jupyter

1. Install `jupyterlab` and [`nb_conda_kernels`](https://github.com/Anaconda-Platform/nb_conda_kernels) using `conda` on the cluster.

    I recommend installing them into their own environment. Let’s call it `jupyter`.
    ```bash
    conda create -n jupyter -c conda-forge jupyterlab nb_conda_kernels
    ```

    _But_ if you're **not** concerned about a package in your _base_ environment conflicting with jupyter or its dependencies now or in the future, you can just install them in your _base_ environment, instead.
    ```bash
    conda install -n base -c conda-forge jupyterlab nb_conda_kernels
    ```
2. If you installed jupyter into its own environment (as recommended), you’ll need to activate that environment every time you want to use it. Do this now.
    ```bash
    conda activate jupyter
    ```
3. [Create a password](https://agent-jay.github.io/2018/03/jupyterserver/#step-1-password-setup)
    ```bash
    jupyter server --generate-config
    jupyter server password
    ```

## Setup a conda environment to use with jupyter
**Scenario**: You would like to install R version 4.0 and use it from within Jupyter.
In order to access your new R installation from within Jupyter, you will also need to install [IRKernel](https://github.com/IRkernel/IRkernel). So, create a new environment called r4 containing both R version 4.0 and IRKernel:
```bash
conda create -n r4 -c conda-forge 'r-base=4.0' 'r-irkernel'
```

**Scenario**: A similar strategy can be employed for Python. Let’s say you’d like to install python 3.5 in a new conda environment called py35, and you want to run your python 3.5 installation from within Jupyter.
To do this, just install both python 3.5 and [IPython kernel](https://ipython.readthedocs.io/en/stable/install/kernel_install.html) in a new environment:
```bash
conda create -n py35 -c conda-forge 'python=3.5' ipykernel
```

**Scenario**: Instead, you have an existing environment called py35 with python 3.5, and you’d like to run your python 3.5 installation from within Jupyter.
Just install [IPython kernel](https://ipython.readthedocs.io/en/stable/install/kernel_install.html) in the existing environment:
```bash
conda install -n py35 -c conda-forge ipykernel
```

<p style='font-size:x-small'>In general, you will need to install a new kernel in every conda environment that you would like to use with jupyter. But you do not need to install a new copy of jupyter in every environment. The kernels that you install will allow jupyter in your `jupyter` environment to execute each version of Python and R that you install in other environments. And the `nb_conda_kernels` package will automatically detect kernels you’ve installed in those environments on startup.</p>

## Usage

1. When ssh-ing into the cluster, make sure to also [add a port-forward](https://www.digitalocean.com/community/tutorials/how-to-set-up-jupyter-notebook-for-python-3#step-2-(optional)-%E2%80%94-using-ssh-tunneling-to-connect-to-a-server-installation) to your ssh command. This will connect a port on your computer (ex: `8888`) to a port on the cluster (ex: `9999`):
    ```bash
    ssh -L 8888:localhost:9999 your_server_username@your_server_address
    ```
    It’s possible for you to choose a port that is being used by somebody else on the cluster. Pick a port 1024 < x < 65535 that nobody else is using. So **don’t use `9999`**.
2. If you installed jupyter into its own environment, activate that environment on the cluster:
    ```bash
    conda activate jupyter
    ```
3. On the cluster, start jupyter with the cluster port you used in step #1. And don’t let jupyter try to open a browser:
```bash
jupyter lab --no-browser --port 9999
```
4. In a browser running on your own computer, go to http://localhost:8888/lab
5. Log in using the password you created in step #3 of the setup
6. Your R and python installations/environments should appear in the Notebook Launcher menu. Click on one to create a notebook. Watch [this video](https://www.youtube.com/watch?v=A5YyoCKxEOU&t=112s) to learn how to write one.

When you’re done, make sure to shut down both your notebook kernel(s) (“Kernel” > “Shut Down All Kernels”) and Jupyter itself (“File” > “Shutdown”) from the menu bar! Otherwise, they might continue running on the cluster forever, especially if you use `screen` sessions.

## Usage on an HPC
If you are working on a high performance computing cluster like TSCC or Expanse, you should perform the following steps between steps 1 and 2 from above.

1. Start an interactive node on your computing cluster
2. Reverse-forward the port you chose (from step #1 above) from your interactive node back to the login node.

    On TSCC (with the PBS-torque engine):
    ```bash
    ssh -NfR 9999:localhost:9999 $PBS_O_HOST
    ```
    
    **OR** on Expanse (with the SLURM engine):
    ```bash
    ssh -NfR 9999:localhost:9999 $SLURM_SUBMIT_HOST
    ```

<!-- ## Usage on Expanse
TODO -->

-------
You're done! The rest of this document is optional.

## Simplified Usage Setup (optional)
It’s possible to reduce the usage instructions above to just three memorable steps (see the next section below). Just follow these recommendations for a bit of extra setup.

### Remove the port-forwarding step
You can set up ssh so that it creates a port forward by default, whenever you connect to the cluster. Just add the `LocalForward` directive to the `~/.ssh/config` on your computer like [this](https://www.digitalocean.com/community/tutorials/how-to-configure-custom-connection-options-for-your-ssh-client#connection-forwarding). Here’s an example of what you can put in your `~/.ssh/config`:
```
Host a_short_server_nickname
    HostName your_server_address
    User your_server_username
    LocalForward 8888 localhost:9999
```

### Remove the arguments that you pass to Jupyter
You can have jupyter use the arguments above by default, whenever it is executed. Just set them in [the `~/.jupyter/jupyter_server_config.py` config file](https://jupyter-server.readthedocs.io/en/latest/other/full-config.html), similar to what they do [here](https://jupyter-notebook.readthedocs.io/en/stable/public_server.html#running-a-public-notebook-server):
1. [Change the default port](https://stackoverflow.com/a/40436673) from this
    ```
    ## The port the server will listen on (env: JUPYTER_PORT).
    #  Default: 0
    # c.ServerApp.port = 0
    ```
    to this
    ```
    ## The port the server will listen on (env: JUPYTER_PORT).
    #  Default: 0
    c.ServerApp.port = 9999
    ```
2. [Don’t open a browser](https://stackoverflow.com/a/52210809). This should be _False_ by default:
    ```
    ## Whether to open in a browser after starting.
    #                          The specific browser used is platform dependent and
    #                          determined by the python standard library `webbrowser`
    #                          module, unless it is overridden using the --browser
    #                          (ServerApp.browser) configuration option.
    #  Default: False
    # c.ServerApp.open_browser = False
    ```

### Do other things in your terminal while Jupyter is running
To prevent jupyter from barfing into your terminal when it starts running, you can [send the start-up command to the background](https://unix.stackexchange.com/a/70728) and [silence it](https://unix.stackexchange.com/a/80632) in step #3 of the usage section:
```bash
jupyter lab &>/dev/null &
```

### Activate the jupyter environment and execute jupyter simultaneously
For those who installed jupyter into a separate jupyter conda environment, you can actually activate the environment and execute the command simultaneously, in a single command:
```bash
conda run -n jupyter jupyter lab
```

### Create an alias
You can also create an [alias](https://linuxize.com/post/how-to-create-bash-aliases/) called `jup` so that you don’t have to type out the full command to start `jupyter` every time. Just run this command to add the alias to your `.bashrc` file on the cluster:
```bash
echo 'alias jup="conda run -n jupyter jupyter lab &>/dev/null"' >> ~/.bashrc
```
Then, close and reopen your terminal or ssh connection.

## Simplified Usage (optional)

Assuming you followed the steps to setup the simplified usage, the usage instructions will reduce to just three (or four) memorable steps:
1. ssh into the cluster
    ```bash
    ssh your_short_server_nickname
    ```
2. If you are working on an HPC, you can login to an interactive node and reverse-forward your port at this stage.
3. Start jupyter on the cluster
    ```bash
    jup &
    ```
4. In a browser running on your own computer, go to http://localhost:8888/lab

## Use screen to keep Jupyter running even with bad internet!
If your ssh connection dies because of a choppy internet connection, your Jupyter instance will die, too. You can run jupyter from within a screen session to make it persist. Let’s adapt the Simplified Usage instructions.

1. ssh into the cluster
    ```bash
    ssh your_short_server_nickname
    ```
2. Create and enter a screen session named jupyter
    ```bash
    screen -S jupyter
    ```
3. View a list of running screen sessions
    ```bash
    screen -ls
    ```
4. If you are working on an HPC, you can login to an interactive node and reverse-forward your port at this stage.
5. Start jupyter on the cluster
    ```bash
    jup &
    ```
6. In a browser running on your own computer, go to http://localhost:8888/lab
7. If your ssh connection dies, everything in your screen session will continue running. Just log back in and reattach the session to continue where you left off. (Note: If you are working on an HPC, you will need to use the same login node as before!)
    ```bash
    screen -r jupyter
    ```
8. You can also terminate a session while inside of it
    ```bash
    exit
    ```
    Or, if you’d like to detach from a session (instead of terminating it) while you’re inside of it, press ctrl+a and then the letter d